# Hardbrücke 

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import plotly.express as px

In [2]:
hb = pd.read_csv('frequenzen_hardbruecke_2020.csv')

In [3]:
hb['Timestamp'] = pd.to_datetime(hb['Timestamp'])

In [4]:
hb['hour'] = hb['Timestamp'].dt.hour

In [5]:
hb.head()

,In,Out,Timestamp,Name,hour
0,1,0,2020-01-01 23:55:00,Ost-Nord total,23
1,2,5,2020-01-01 23:50:00,Ost-Nord total,23
2,1,0,2020-01-01 23:45:00,Ost-Nord total,23
3,1,4,2020-01-01 23:40:00,Ost-Nord total,23
4,1,0,2020-01-01 23:35:00,Ost-Nord total,23


In [6]:
hb.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 714681 entries, 0 to 714680
Data columns (total 5 columns):
 #   Column     Non-Null Count   Dtype         
---  ------     --------------   -----         
 0   In         714681 non-null  int64         
 1   Out        714681 non-null  int64         
 2   Timestamp  714681 non-null  datetime64[ns]
 3   Name       714681 non-null  object        
 4   hour       714681 non-null  int64         
dtypes: datetime64[ns](1), int64(3), object(1)
memory usage: 27.3+ MB


In [7]:
hb.describe()

,In,Out,hour
count,714681.000000,714681.000000,714681.000000
mean,12.405247,10.299464,12.853272
std,16.061613,12.936968,6.397771
min,0.000000,0.000000,0.000000
25%,3.000000,2.000000,8.000000
50%,7.000000,6.000000,13.000000
75%,16.000000,13.000000,18.000000
max,253.000000,276.000000,23.000000


In [15]:
hb2 = hb.set_index(['Timestamp','Name','hour']).stack().reset_index()

In [17]:
hb2.head()

,Timestamp,Name,hour,level_3,0
0,2020-01-01 23:55:00,Ost-Nord total,23,In,1
1,2020-01-01 23:55:00,Ost-Nord total,23,Out,0
2,2020-01-01 23:50:00,Ost-Nord total,23,In,2
3,2020-01-01 23:50:00,Ost-Nord total,23,Out,5
4,2020-01-01 23:45:00,Ost-Nord total,23,In,1


In [16]:
px.box(hb2,x=0,)

,In,Out,Timestamp,Name,hour
0,1,0,2020-01-01 23:55:00,Ost-Nord total,23
1,2,5,2020-01-01 23:50:00,Ost-Nord total,23
2,1,0,2020-01-01 23:45:00,Ost-Nord total,23
3,1,4,2020-01-01 23:40:00,Ost-Nord total,23
4,1,0,2020-01-01 23:35:00,Ost-Nord total,23
...,...,...,...,...,...
714676,1,6,2020-12-31 00:25:00,West-VBZ total,0
714677,0,10,2020-12-31 00:20:00,West-VBZ total,0
714678,4,4,2020-12-31 00:15:00,West-VBZ total,0
714679,3,5,2020-12-31 00:10:00,West-VBZ total,0
